In [750]:
#Model 1

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import joblib


In [3]:
df = pd.read_csv('weather.csv')

In [4]:
unique_provinces = df['province'].unique()
num_provinces = len(unique_provinces)

# Mã hóa OneHotEncoder cho các tỉnh thành
df_province = pd.DataFrame({'province': unique_provinces})
onehot_encoder = OneHotEncoder(sparse_output=False)
encoded_provinces = onehot_encoder.fit_transform(df_province[['province']])

df_province_encoded = pd.DataFrame(encoded_provinces, columns=onehot_encoder.get_feature_names_out(['province']))

# Thêm các cột mã hóa vào DataFrame gốc
df = pd.concat([df.reset_index(drop=True), df_province_encoded.reset_index(drop=True)], axis=1)

In [5]:
# Xác định có mưa
df['have_rain'] = df['rain'] > 0

df['have_rain'] = df['have_rain'].astype(bool)

# Kiểm tra phân tích hướng gió
unique_wind_d = df['wind_d'].unique()
num_wind_d = len(unique_wind_d)

# Định dạng hướng gió
wind_direction_mapping = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5
}

df['wind_degree'] = df['wind_d'].map(wind_direction_mapping)

df['date'] = pd.to_datetime(df['date'])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_name()  
df['week_of_year'] = df['date'].dt.isocalendar().week 
df['year_quarter'] = df['date'].dt.to_period('Q').astype('int')
df['month_period'] = df['date'].dt.to_period('M').astype('int')

day_of_week_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

df['day_of_week_encoded'] = df['day_of_week'].map(day_of_week_mapping)


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181960 entries, 0 to 181959
Data columns (total 60 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   province                   181960 non-null  object        
 1   max                        181960 non-null  int64         
 2   min                        181960 non-null  int64         
 3   wind                       181960 non-null  int64         
 4   wind_d                     181960 non-null  object        
 5   rain                       181960 non-null  float64       
 6   humidi                     181960 non-null  int64         
 7   cloud                      181960 non-null  int64         
 8   pressure                   181960 non-null  int64         
 9   date                       181960 non-null  datetime64[ns]
 10  province_Bac Lieu          40 non-null      float64       
 11  province_Ben Tre           40 non-null      float64 

In [755]:
# Thay đổi X để sử dụng cột đã mã hóa
X = df[['province', 'year', 'month', 'day', 'day_of_week_encoded', 'week_of_year', 'year_quarter', 'month_period']]
y = df[['max', 'min', 'wind', 'wind_degree', 'humidi', 'cloud', 'pressure']]

In [756]:
# Mã hóa tỉnh thành phố
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X[['province']])
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(['province']))

In [757]:
# Kết hợp các cột đã mã hóa với dữ liệu ngày tháng
X_final = pd.concat([X.drop('province', axis=1), X_encoded_df], axis=1)

In [758]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42)

In [759]:
# Tiền xử lý dữ liệu (chuẩn hóa)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [760]:
model = MultiOutputRegressor(RandomForestRegressor(random_state=42))

# Huấn luyện mô hình
model.fit(X_train_scaled, y_train)

# Lưu mô hình tốt nhất
model1 = model


In [761]:
# Dự đoán trên tập kiểm tra
y_pred = model1.predict(X_test_scaled)

In [783]:
# Đánh giá mô hình
print('Max Temperature - MSE:', mean_squared_error(y_test['max'], y_pred[:, 0]))
print('Max Temperature - R^2:', r2_score(y_test['max'], y_pred[:, 0]))
print('Min Temperature - MSE:', mean_squared_error(y_test['min'], y_pred[:, 1]))
print('Min Temperature - R^2:', r2_score(y_test['min'], y_pred[:, 1]))
print('Wind - MSE:', mean_squared_error(y_test['wind'], y_pred[:, 2]))
print('Wind - R^2:', r2_score(y_test['wind'], y_pred[:, 2]))
print('Wind Degree - MSE:', mean_squared_error(y_test['wind_degree'], y_pred[:, 3]))
print('Wind Degree - R^2:', r2_score(y_test['wind_degree'], y_pred[:, 3]))
print('Humidity - MSE:', mean_squared_error(y_test['humidi'], y_pred[:, 4]))
print('Humidity - R^2:', r2_score(y_test['humidi'], y_pred[:, 4]))
print('Cloud - MSE:', mean_squared_error(y_test['cloud'], y_pred[:, 5]))
print('Cloud - R^2:', r2_score(y_test['cloud'], y_pred[:, 5]))
print('Pressure - MSE:', mean_squared_error(y_test['pressure'], y_pred[:, 6]))
print('Pressure - R^2:', r2_score(y_test['pressure'], y_pred[:, 6]))

Max Temperature - MSE: 2.9173087968051585
Max Temperature - R^2: 0.8601165042727765
Min Temperature - MSE: 1.5133661592291345
Min Temperature - R^2: 0.9025337422519619
Wind - MSE: 7.892598212061258
Wind - R^2: 0.7184520592256866
Wind Degree - MSE: 1588.7784102962191
Wind Degree - R^2: 0.6855485355884514
Humidity - MSE: 20.88738897559903
Humidity - R^2: 0.7586214902381826
Cloud - MSE: 220.5018375833517
Cloud - R^2: 0.6153468659893951
Pressure - MSE: 2.397195276434381
Pressure - R^2: 0.8885644279650023


In [ ]:
# Tạo DataFrame cho ngày cụ thể
data = {
    'province': ['Ho Chi Minh City'],
    'date': [pd.to_datetime('2024-09-20')],
}

df_input = pd.DataFrame(data)


# Tiền xử lý dữ liệu đầu vào
df_input['year'] = df_input['date'].dt.year
df_input['month'] = df_input['date'].dt.month
df_input['day'] = df_input['date'].dt.day
df_input['day_of_week'] = df_input['date'].dt.day_name()
df_input['week_of_year'] = df_input['date'].dt.isocalendar().week
df_input['year_quarter'] = df_input['date'].dt.to_period('Q').astype('int')
df_input['month_period'] = df_input['date'].dt.to_period('M').astype('int')

day_of_week_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

df_input['day_of_week_encoded'] = df_input['day_of_week'].map(day_of_week_mapping)

# Giả sử encoder đã được khởi tạo trước đó như sau:
# encoder = OneHotEncoder(sparse=False)

# Mã hóa tỉnh thành
province_encoded = encoder.transform(df_input[['province']])
province_encoded_df = pd.DataFrame(province_encoded, columns=encoder.get_feature_names_out(['province']))

# Kết hợp các cột đã mã hóa với dữ liệu ngày tháng
X_input = pd.concat([df_input.drop(['province', 'date', 'day_of_week'], axis=1), province_encoded_df], axis=1)

# Đảm bảo thứ tự cột đúng bằng cách sắp xếp lại theo thứ tự của X_train
X_input = X_input.reindex(columns=X_final.columns, fill_value=0)

# Chuẩn hóa dữ liệu đầu vào
X_input_scaled = scaler.transform(X_input)

# Dự đoán
y_pred_input = model.predict(X_input_scaled)

# In kết quả dự đoán
print('Dự đoán cho Ho Chi Minh City vào ngày 2024-09-20:')
print('Max Temperature:', y_pred_input[0, 0])
print('Min Temperature:', y_pred_input[0, 1])
print('Wind Speed:', y_pred_input[0, 2])
print('Wind Degree:', y_pred_input[0, 3])
print('Humidity:', y_pred_input[0, 4])
print('Cloud:', y_pred_input[0, 5])
print('Pressure:', y_pred_input[0, 6])


In [764]:
# model 2

In [6]:
# Xác định biến đầu vào (X) và đầu ra (y) cho mô hình dự đoán mưa
X_rain = df[['max', 'min', 'wind', 'wind_degree', 'humidi', 'cloud', 'pressure']]
y_rain = df['have_rain']

In [7]:
X_train_rain, X_test_rain, y_train_rain, y_test_rain = train_test_split(X_rain, y_rain, test_size=0.3, random_state=42)

In [8]:
scaler_rain = StandardScaler()
X_train_rain_scaled = scaler_rain.fit_transform(X_train_rain)
X_test_rain_scaled = scaler_rain.transform(X_test_rain)

joblib.dump(scaler_rain, 'scaler_rain.pkl')

['scaler_rain.pkl']

In [9]:
# Định nghĩa tham số cho GridSearch
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Khởi tạo mô hình hồi quy logistic
model2 = RandomForestClassifier(random_state=42)

# Thực hiện GridSearch
grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, cv=2, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_rain_scaled, y_train_rain)

# Huấn luyện mô hình
model2 = grid_search.best_estimator_

In [10]:
# Dự đoán trên tập kiểm tra
y_pred_rain = model2.predict(X_test_rain_scaled)

In [11]:
# Đánh giá mô hình
accuracy = accuracy_score(y_test_rain, y_pred_rain)
f1 = f1_score(y_test_rain, y_pred_rain)
confusion = confusion_matrix(y_test_rain, y_pred_rain)

print('Accuracy:', accuracy)
print('F1 Score:', f1)
print('Confusion Matrix:\n', confusion)

Accuracy: 0.9211914706528908
F1 Score: 0.9497371188222923
Confusion Matrix:
 [[ 9642  2660]
 [ 1642 40644]]


In [ ]:
# Giả sử y_pred_input đã được tính toán từ model 1
max_temp = y_pred_input[0, 0]
min_temp = y_pred_input[0, 1]
wind_speed = y_pred_input[0, 2]
wind_degree = y_pred_input[0, 3]
humidity = y_pred_input[0, 4]
cloud = y_pred_input[0, 5]
pressure = y_pred_input[0, 6]

# Tạo DataFrame cho đầu vào của model 2
data_model2 = {
    'max': [max_temp],
    'min': [min_temp],
    'wind': [wind_speed],
    'wind_degree': [wind_degree],
    'humidi': [humidity],
    'cloud': [cloud],
    'pressure': [pressure]
}

df_input_model2 = pd.DataFrame(data_model2)

# Nếu model 2 cần chuẩn hóa đầu vào
X_input_model2_scaled = scaler_rain.transform(df_input_model2)

# Dự đoán với model 2
y_pred_rain = model2.predict(X_input_model2_scaled)

# In kết quả dự đoán của model 2
print('Dự đoán về khả năng có mưa cho Ho Chi Minh City vào ngày 2024-09-20:')
print('Có mưa:', y_pred_rain[0])

In [771]:
# Model 3

In [12]:
# Xác định biến đầu vào (X) và đầu ra (y) cho mô hình dự đoán lượng mưa
df_rain_filtered = df[df['have_rain'] == True]

df_rain_filtered.info()

X_rain_amount = df_rain_filtered[['max', 'min', 'wind', 'wind_degree', 'humidi', 'cloud', 'pressure']]
y_rain_amount = df_rain_filtered['rain']

<class 'pandas.core.frame.DataFrame'>
Index: 141220 entries, 0 to 181959
Data columns (total 60 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   province                   141220 non-null  object        
 1   max                        141220 non-null  int64         
 2   min                        141220 non-null  int64         
 3   wind                       141220 non-null  int64         
 4   wind_d                     141220 non-null  object        
 5   rain                       141220 non-null  float64       
 6   humidi                     141220 non-null  int64         
 7   cloud                      141220 non-null  int64         
 8   pressure                   141220 non-null  int64         
 9   date                       141220 non-null  datetime64[ns]
 10  province_Bac Lieu          14 non-null      float64       
 11  province_Ben Tre           14 non-null      float64      

In [13]:
X_train_rain_amount, X_test_rain_amount, y_train_rain_amount, y_test_rain_amount = train_test_split(X_rain_amount, y_rain_amount, test_size=0.3, random_state=42)

In [14]:
scaler_rain_amount = StandardScaler()
X_train_rain_amount_scaled = scaler_rain_amount.fit_transform(X_train_rain_amount)
X_test_rain_amount_scaled = scaler_rain_amount.transform(X_test_rain_amount)
joblib.dump(scaler_rain_amount, 'scaler_rain_amount.pkl')

['scaler_rain_amount.pkl']

In [15]:
# Định nghĩa tham số cho GridSearch
param_grid = {
    'n_estimators': [50, 100],  # Số lượng cây trong rừng
    'max_depth': [None, 5],  # Độ sâu tối đa của mỗi cây
    'min_samples_split': [2, 5],  # Số lượng mẫu tối thiểu để chia
    'min_samples_leaf': [1, 2, ],  # Số lượng mẫu tối thiểu trong mỗi lá
    'max_features': ['sqrt', 'log2']  # Số lượng biến tối đa để xem xét tại mỗi phân nhánh
}

# Tạo mô hình Random Forest
model = RandomForestRegressor(random_state=42)

# Thực hiện GridSearch
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_rain_amount_scaled, y_train_rain_amount)

# Lưu mô hình tốt nhất
model3 = grid_search.best_estimator_


d:\python\weather-prediction\env\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [16]:
# # Dự đoán mẫu
y_pred_rain_amount = model3.predict(X_test_rain_amount_scaled)

In [17]:
#Đánh giá mô hình 
mse_rain_amount = mean_squared_error(y_test_rain_amount, y_pred_rain_amount)
r2_rain_amount = r2_score(y_test_rain_amount, y_pred_rain_amount)

print('MSE:', mse_rain_amount)
print('R²:', r2_rain_amount)

MSE: 92.82441159683792
R²: 0.5661444045024135


In [ ]:
# Kiểm tra kết quả dự đoán của model 2
if y_pred_rain[0] == 1:  # Nếu có mưa
    # Tạo DataFrame cho đầu vào của model 3 (dự đoán lượng mưa)
    data_model3 = {
        'max': [max_temp],
        'min': [min_temp],
        'wind': [wind_speed],
        'wind_degree': [wind_degree],
        'humidi': [humidity],
        'cloud': [cloud],
        'pressure': [pressure]
    }

    df_input_model3 = pd.DataFrame(data_model3)

    # Nếu model 3 cần chuẩn hóa đầu vào
    X_input_model3_scaled = scaler_rain_amount.transform(df_input_model3)

    # Dự đoán với model 3
    y_pred_rain_amount = model3.predict(X_input_model3_scaled)

    # In kết quả dự đoán lượng mưa
    print('Dự đoán lượng mưa cho Ho Chi Minh City vào ngày 2024-09-20:')
    print('Lượng mưa dự đoán:', y_pred_rain_amount[0])  # Giá trị lượng mưa
else:
    print('Dự đoán cho Ho Chi Minh City vào ngày 2024-09-20: Không có mưa.')


In [ ]:
import joblib

joblib.dump(model1, 'weather_model.pkl')
joblib.dump(model2, 'weather_model_rain.pkl')
joblib.dump(model3, 'weather_model_rain_amount.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(encoder, 'encoder.pkl')
joblib.dump(X_final, "X_final.pkl")